In [1]:
# 네이버 검색 API 예제 - 블로그 검색
# 출처: https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EB%B8%94%EB%A1%9C%EA%B7%B8

import time
import os
import sys
import json
import re
import urllib.request
import openai
import random
from utils import generate_text_davinci, generate_text_GPT2, NaverSearchAPI, load_embed_model

client_id = ""
client_secret = ""
openai.api_key = "sk-"# **GPT  key 지정

naver_api = NaverSearchAPI(client_id=client_id, client_secret=client_secret)

# 임베딩 모델 로딩
#WORD_EMBDDING_MODEL1, BI_ENCODER1 = load_embed_model('./model/kpf-sbert-v1.1', 'mean', 768, "cpu")


In [5]:
start_time = time.time()

query = "제주도 겨울에 갈만한 관광치 추천"  
#contexts, error = naver_api.search_naver_ex(query=query, classification=['news','webkr'], display=10, top_k=10, bi_encoder=BI_ENCODER1)

# 네이버 검색
#classification=['news', 'webkr', 'blog']
classification=['news', 'webkr', 'blog']
 # 랜덤하게 2개 선택
#selected_items = random.sample(classification, 2)
#random.shuffle(classification) # 리스트에 있는 것을 섞음

#start = random.randint(1, 3)
start = 1
print(f'classification:{classification}/start:{start}')

contexts, best_contexts, error = naver_api.search_naver(query=query, classification=classification, 
                                         start=start, display=6)
        
# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}\n')

classification:['news', 'webkr', 'blog']/start:1
time:0.71



In [6]:
context:str = ""
text:str = ""
for con in contexts:
    context += con['title']+'\n'+con['descript']+'\n\n'
    text += con['title']+'\n'+ con['descript'] + '\n'+con['link']+'\n('+con['score']+')\n\n'
print(text)

12월 겨울 제주도 여행 코스 가볼만한곳 명소 추천 BEST 10 :: 로띠스마
이번 글에서는 12월 겨울 제주도 여행 코스 가볼만한곳 명소 추천 BEST 10에 대해 알아보겠다. 겨울 제주도 여행에서 놓치지 말아야 할 것이 있다면 바로 한라산의 설경이다. 정상에 소복하게 쌓인 눈과 나뭇가지에 하얗게 핀 눈꽃이 가득한 한라산의 경치는 신비로우면서도 경이롭다. 화사한 동백꽃이 만발한 카멜리아힐도 또 다른 겨울 명소라고 할 수 있다...
https://lotisma.tistory.com/2746
(0)

겨울 서귀포 가볼만한곳 10곳
있으며 제주도의 서쪽 해안인 애월을 즐기는 제주 올레 15·B 코스는 안전한... 앙상한 가지만 남는 겨울에도 초록세상을 걸어볼 수 있어 겨울 제주도 서귀포 가볼만한곳 10곳으로 추천하는 명소입니다. 녹차 밭을...
https://ohnovember.tistory.com/entry/%EA%B2%A8%EC%9A%B8-%EC%84%9C%EA%B7%80%ED%8F%AC-%EA%B0%80%EB%B3%BC%EB%A7%8C%ED%95%9C%EA%B3%B3-10%EA%B3%B3
(0)

제주도 겨울 여행코스 추천 7곳, 춥지만 오히려 좋아!
제주도 겨울 여행, 겨울 제주도 가볼만한곳, 제주도 여행지 추천 | 겨울 여행지를 고심하기란 쉽지 않습니다. 추위는 이불 밖으로 나가는 것을 방해하는 최대 요소이기 때문에 어떤 행선지를 정할지 고민되기 마련인데요. 이럴 때, 제주도로 시선을 돌려보세요. 제주는 자연의 아름다운 경치와 제주에서만 맛볼 수 있는 먹거리, 탁월한 볼거리와 즐길거리가 마련되어...
https://brunch.co.kr/@hotelscomkr/1362
(0)

제주 겨울명소 21곳 총정리 (제주도 겨울에 가볼만한 곳)
제주 겨울명소 21곳 총정리 (제주도 겨울에 가볼만한 곳) 올해 보았던 드라마 중에서 가장 기억이 남는... 부는 제주도를 굳이 겨울에 갈 이유가 있냐고 묻는 분들이 있다면, 아직 겨울 산행이나 겨울 제주도의... 화..

In [7]:
start_time = time.time()

prompt = f'{context}\n\nQ:{query}?.요약해줘\nA:'
print(prompt)    
    
response = generate_text_GPT2(gpt_model="gpt-3.5-turbo-1106",
                              prompt=prompt, timeout=10, stream=True)

# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}\n')

print(f'답변:{response[0]}')

12월 겨울 제주도 여행 코스 가볼만한곳 명소 추천 BEST 10 :: 로띠스마
이번 글에서는 12월 겨울 제주도 여행 코스 가볼만한곳 명소 추천 BEST 10에 대해 알아보겠다. 겨울 제주도 여행에서 놓치지 말아야 할 것이 있다면 바로 한라산의 설경이다. 정상에 소복하게 쌓인 눈과 나뭇가지에 하얗게 핀 눈꽃이 가득한 한라산의 경치는 신비로우면서도 경이롭다. 화사한 동백꽃이 만발한 카멜리아힐도 또 다른 겨울 명소라고 할 수 있다...

겨울 서귀포 가볼만한곳 10곳
있으며 제주도의 서쪽 해안인 애월을 즐기는 제주 올레 15·B 코스는 안전한... 앙상한 가지만 남는 겨울에도 초록세상을 걸어볼 수 있어 겨울 제주도 서귀포 가볼만한곳 10곳으로 추천하는 명소입니다. 녹차 밭을...

제주도 겨울 여행코스 추천 7곳, 춥지만 오히려 좋아!
제주도 겨울 여행, 겨울 제주도 가볼만한곳, 제주도 여행지 추천 | 겨울 여행지를 고심하기란 쉽지 않습니다. 추위는 이불 밖으로 나가는 것을 방해하는 최대 요소이기 때문에 어떤 행선지를 정할지 고민되기 마련인데요. 이럴 때, 제주도로 시선을 돌려보세요. 제주는 자연의 아름다운 경치와 제주에서만 맛볼 수 있는 먹거리, 탁월한 볼거리와 즐길거리가 마련되어...

제주 겨울명소 21곳 총정리 (제주도 겨울에 가볼만한 곳)
제주 겨울명소 21곳 총정리 (제주도 겨울에 가볼만한 곳) 올해 보았던 드라마 중에서 가장 기억이 남는... 부는 제주도를 굳이 겨울에 갈 이유가 있냐고 묻는 분들이 있다면, 아직 겨울 산행이나 겨울 제주도의... 화...

소소한 ‘제주여행’ 꿀팁! 〈겨울편〉> 여행기사 :대한민국 구석구석
‘초저녁에 외롭게 떠 있는 샛별 같다’ 해서 ‘새별’이라는 예쁜 이름이 붙은 오름인 새별오름은 봄, 여름보다는 가을, 겨울에 여행자들이 가장 많이 찾는 오름 중 하나입니다. 제주시에서 서귀포로 가는 서부산업도로를 따라 달리다 보면 벌판에 동그랗게 능선이 살아있는 것처럼 보이는 오름이 바로 새별오름인데요, 오르는 길의 경사도가 만만